In [1]:
import os
import pickle
import pandas as pd
import numpy as np

## Load filtered DF

In [2]:
Cleaned_df = pd.read_pickle("cleaned_df_with_id_without_encode.pkl")
Cleaned_df.shape

(6954, 17)

In [3]:
Cleaned_df['id']

0            958
1           5858
2           7918
3           8142
5           8739
          ...   
7269    44758668
7270    44799193
7271    44805385
7272    44810526
7273    44811730
Name: id, Length: 6954, dtype: int64

## Avgering sentiment score

In [4]:
df_sentiment = pd.read_pickle("../../Tanny/df_sentiment.pkl")
df_senti = df_sentiment[['listing_id', 'sentiment_compound']]
df_senti= df_senti.groupby('listing_id', as_index=False)['sentiment_compound'].mean()
df_senti

,listing_id,sentiment_compound
0,958,0.891257
1,5858,0.913694
2,7918,0.721942
3,8142,0.907329
4,8339,0.970007
...,...,...
5529,44280619,0.811250
5530,44379576,0.774225
5531,44382961,-0.697200
5532,44394083,0.927300


## Merge sentimentscore and safe index score

In [5]:
df_w_sentiment = pd.merge(left=Cleaned_df , right=df_senti, left_on='id', right_on='listing_id').drop(columns=['listing_id','id'])
df_w_sentiment.reset_index(drop=True, inplace=True)
df_w_sentiment

,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,amenities,price,minimum_nights,review_scores_rating,reviews_per_month,price_log,sentiment_compound
0,within an hour,1.00,0.99,1,1,Western Addition,Entire apartment,Entire home/apt,3,1.0,"[""Heating"", ""Hot water"", ""Stove"", ""Iron"", ""Dry...",132.0,2,97.0,1.91,4.882802,0.891257
1,within a day,0.60,0.83,0,2,Bernal Heights,Entire apartment,Entire home/apt,5,1.0,"[""Smoke alarm"", ""Heating"", ""Kitchen"", ""First a...",235.0,30,98.0,0.81,5.459586,0.913694
2,within a few hours,1.00,1.00,0,10,Haight Ashbury,Private room in apartment,Private room,2,4.0,"[""Host greets you"", ""Heating"", ""Hot water"", ""K...",56.0,32,84.0,0.14,4.025352,0.721942
3,within a few hours,1.00,1.00,0,10,Haight Ashbury,Private room in apartment,Private room,2,4.0,"[""Host greets you"", ""Heating"", ""Hot water"", ""K...",56.0,32,93.0,0.11,4.025352,0.907329
4,within an hour,1.00,0.96,1,2,Mission,Private room in condominium,Private room,3,1.0,"[""Heating"", ""Hot water"", ""Children\u2019s book...",169.0,1,98.0,5.47,5.129899,0.922009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291,within an hour,1.00,0.97,0,4,Bayview,Entire house,Entire home/apt,10,1.5,"[""Smoke alarm"", ""Hot water"", ""Kitchen"", ""Fire ...",299.0,1,100.0,1.00,5.700444,0.886000
5292,within a few hours,1.00,0.71,0,0,Inner Richmond,Entire apartment,Entire home/apt,2,1.0,"[""Heating"", ""Hot water"", ""Stove"", ""Air conditi...",150.0,1,100.0,2.00,5.010635,0.811250
5293,within an hour,1.00,0.80,0,1,Outer Mission,Entire house,Entire home/apt,8,1.0,"[""Heating"", ""Hot water"", ""Stove"", ""Iron"", ""Car...",145.0,2,60.0,1.00,4.976734,-0.697200
5294,within an hour,1.00,0.85,0,0,Parkside,Entire house,Entire home/apt,8,2.0,"[""Heating"", ""Hot water"", ""Kitchen"", ""Wifi"", ""P...",195.0,1,100.0,1.00,5.273000,0.927300


## Merge listing and safe index score

In [6]:
safe_df = pd.read_pickle("../../SafeIndex/safeIndex_normalized.pkl")
safe_df = safe_df[safe_df['Category'] == 'overall']
safe_df.rename(columns={'Normalized_score':'SafeIndex_Score'}, inplace=True)
safe_df.shape

(34, 3)

In [7]:
df = pd.merge(left=df_w_sentiment , right=safe_df, left_on='neighbourhood_cleansed', right_on='Area').drop(columns=['Area', 'Category'])
df.reset_index(drop=True, inplace=True)
df

,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,amenities,price,minimum_nights,review_scores_rating,reviews_per_month,price_log,sentiment_compound,SafeIndex_Score
0,within an hour,1.00,0.99,1,1,Western Addition,Entire apartment,Entire home/apt,3,1.0,"[""Heating"", ""Hot water"", ""Stove"", ""Iron"", ""Dry...",132.0,2,97.0,1.91,4.882802,0.891257,0.400000
1,within a few hours,0.97,0.79,1,38,Western Addition,Entire apartment,Entire home/apt,5,2.0,"[""Heating"", ""Hot water"", ""Stove"", ""Iron"", ""Dry...",166.0,30,91.0,0.15,5.111988,0.711287,0.400000
2,within a day,0.50,0.76,1,3,Western Addition,Private room in townhouse,Private room,2,1.0,"[""Heating"", ""Hot water"", ""Stove"", ""Iron"", ""Car...",130.0,2,97.0,3.92,4.867534,0.928239,0.400000
3,within a day,0.50,0.76,1,3,Western Addition,Private room in townhouse,Private room,2,1.0,"[""Heating"", ""Hot water"", ""Stove"", ""Iron"", ""Car...",110.0,2,98.0,3.73,4.700480,0.919248,0.400000
4,within an hour,1.00,1.00,0,2,Western Addition,Entire apartment,Entire home/apt,10,2.0,"[""Heating"", ""Children\u2019s books and toys"", ...",480.0,3,91.0,0.32,6.173786,0.832521,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,within an hour,1.00,0.99,1,1,Visitacion Valley,Entire townhouse,Entire home/apt,5,1.0,"[""Heating"", ""Hot water"", ""Stove"", ""Iron"", ""Dry...",121.0,2,100.0,0.13,4.795791,0.865800,0.581538
5289,within an hour,1.00,1.00,1,1,Visitacion Valley,Entire guesthouse,Entire home/apt,2,1.0,"[""Smoke alarm"", ""Heating"", ""Hot water"", ""Free ...",151.0,2,99.0,4.11,5.017280,0.890267,0.581538
5290,-,0.00,1.00,0,3,Visitacion Valley,Entire guesthouse,Entire home/apt,2,1.0,"[""Heating"", ""Iron"", ""Dryer"", ""Coffee maker"", ""...",179.0,2,100.0,0.47,5.187386,0.938600,0.581538
5291,-,0.00,1.00,0,0,Visitacion Valley,Entire guest suite,Entire home/apt,2,1.0,"[""Heating"", ""Hot water"", ""Iron"", ""Dryer"", ""Cof...",190.0,30,100.0,0.48,5.247024,0.840650,0.581538


###  Remove unncessary features

In [8]:
Filepath = "remove_features_without_encoding.pkl"

if os.path.exists(Filepath):
    df = pd.read_pickle(Filepath)
    print("read success!")
else:
    amentities_col = df.amenities
    amentities_dict = {}
    id = 0
    for amentities in amentities_col:
        x_striped = amentities[1:-1]
        x_arr = x_striped.split(',')

        for amentity in x_arr:
            name = "{}".format((amentity.strip())[1:-1].lower().replace("\\u2019","'"))

            if name not in amentities_dict:
                amentities_dict[name] = 1
            else:
                amentities_dict[name] += 1
    sorted_amentities = dict(sorted(amentities_dict.items(), key=lambda x: x[1], reverse=True))
    sorted_amentities = {k: v for k, v in sorted_amentities.items() if v >= 30}
    filter_amentities = list(sorted_amentities.keys())

    for item in filter_amentities:
        df[item] = 0

    for index, rows in df.iterrows():
        tmp = rows['amenities']
        tmp = (tmp.strip())[1:-1].lower().replace("\\u2019","'").split(",")
        for item in tmp:
            item = item.replace('"', "").strip()
            if item in filter_amentities:
                df.loc[index, item] = 1

    print("Before",df.shape)
    drop_columns = ['hangers', 'carbon monoxide alarm', 'iron', 'shampoo',
                    'laptop-friendly workspace', 'dryer', 'microwave',
                    'dishes and silverware', 'cooking basics', 'stove',
                    'extra pillows and blankets', 'gym', 'bedroom comforts',
                    "children's dinnerware", 'full kitchen', 'self check-in',
                    'outlet covers', 'barbecue utensils', 'baby bath',
                    'table corner guards']
    df.drop(columns =drop_columns, inplace=True)
    print("After",df.shape)
    df.to_pickle(Filepath, protocol=3)

df

Before (5293, 100)
After (5293, 80)


,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,...,game console,stair gates,fireplace guards,pool,pocket wifi,changing table,beach essentials,window guards,ev charger,beachfront
0,within an hour,1.00,0.99,1,1,Western Addition,Entire apartment,Entire home/apt,3,1.0,...,0,0,0,0,0,0,0,0,0,0
1,within a few hours,0.97,0.79,1,38,Western Addition,Entire apartment,Entire home/apt,5,2.0,...,0,0,0,0,0,0,0,0,0,0
2,within a day,0.50,0.76,1,3,Western Addition,Private room in townhouse,Private room,2,1.0,...,0,0,0,0,0,0,0,0,0,0
3,within a day,0.50,0.76,1,3,Western Addition,Private room in townhouse,Private room,2,1.0,...,0,0,0,0,0,0,0,0,0,0
4,within an hour,1.00,1.00,0,2,Western Addition,Entire apartment,Entire home/apt,10,2.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,within an hour,1.00,0.99,1,1,Visitacion Valley,Entire townhouse,Entire home/apt,5,1.0,...,0,0,0,0,0,0,0,0,0,0
5289,within an hour,1.00,1.00,1,1,Visitacion Valley,Entire guesthouse,Entire home/apt,2,1.0,...,0,0,0,0,0,0,0,0,0,0
5290,-,0.00,1.00,0,3,Visitacion Valley,Entire guesthouse,Entire home/apt,2,1.0,...,0,0,0,0,0,0,0,0,0,0
5291,-,0.00,1.00,0,0,Visitacion Valley,Entire guest suite,Entire home/apt,2,1.0,...,0,0,0,0,0,0,0,1,0,0


## Encoding

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
OHE_features = ['host_response_time','property_type','property_type','room_type','neighbourhood_cleansed']
OHE_df = df[OHE_features]
OHE_df

,host_response_time,property_type,property_type,room_type,neighbourhood_cleansed
0,within an hour,Entire apartment,Entire apartment,Entire home/apt,Western Addition
1,within a few hours,Entire apartment,Entire apartment,Entire home/apt,Western Addition
2,within a day,Private room in townhouse,Private room in townhouse,Private room,Western Addition
3,within a day,Private room in townhouse,Private room in townhouse,Private room,Western Addition
4,within an hour,Entire apartment,Entire apartment,Entire home/apt,Western Addition
...,...,...,...,...,...
5288,within an hour,Entire townhouse,Entire townhouse,Entire home/apt,Visitacion Valley
5289,within an hour,Entire guesthouse,Entire guesthouse,Entire home/apt,Visitacion Valley
5290,-,Entire guesthouse,Entire guesthouse,Entire home/apt,Visitacion Valley
5291,-,Entire guest suite,Entire guest suite,Entire home/apt,Visitacion Valley


In [11]:
enc = OneHotEncoder()
enc.fit(OHE_df)
onehotlabels = pd.DataFrame(enc.transform(OHE_df).toarray(), columns =  enc.get_feature_names(OHE_df.columns))
onehotlabels

,host_response_time_-,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,property_type_Earth house,property_type_Entire apartment,property_type_Entire bungalow,property_type_Entire cabin,property_type_Entire condominium,...,neighbourhood_cleansed_Parkside,neighbourhood_cleansed_Potrero Hill,neighbourhood_cleansed_Presidio Heights,neighbourhood_cleansed_Russian Hill,neighbourhood_cleansed_Seacliff,neighbourhood_cleansed_South of Market,neighbourhood_cleansed_Twin Peaks,neighbourhood_cleansed_Visitacion Valley,neighbourhood_cleansed_West of Twin Peaks,neighbourhood_cleansed_Western Addition
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5289,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5290,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5291,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
import pickle
Filepath = "remove_features_with_encoding.pkl"
if os.path.exists(Filepath):
#     mergedDf = pd.read_pickle(Filepath)
    with open(Filepath, 'rb') as f:
        mergedDf = pickle.load(f)
    print("read success!")
else:
    mergedDf = onehotlabels.merge(df, left_index=True, right_index=True)
    mergedDf.to_pickle(Filepath, protocol=3)